# DNA PROBLEM
The high-level solution is presented in thress simple steps:

1) I just show only 4 records (2 FASTA sequences) for a partition, but in reality, each partition may contain thousands or millions of records. If total of all your input is N records and P is the number of partitions, then each partition will have about (N / P) records.

2) If we have enough resources in our Spark cluster, then each partition can be processed in parallel and independently

3) If you have a lot of data, but your requirement is to extract small amount of information from all data, then mapPartitions() might be good choice and will out perform map() and flatMap() transformations.

# Step 1: Create an RDD of String from Input

In [7]:
from pyspark.sql import SparkSession

spark= SparkSession.builder \
    .appName("dna-base-rdd")    \
    .master("local")    \
    .getOrCreate()


input_path = "C:/Users/malam/development/data/spark/sample.fasta"
records_rdd = spark.sparkContext.textFile(input_path)

In [8]:
records_rdd.collect()   

['>seq1',
 'cGTAaccaataaaaaaacaagcttaacctaattc',
 '>seq2',
 'agcttagTTTGGatctggccgggg',
 '>seq3',
 'gcggatttactcCCCCCAAAAANNaggggagagcccagataaatggagtctgtgcgtccaca',
 'gaattcgcacca',
 'AATAAAACCTCACCCAT',
 'agagcccagaatttactcCCC',
 '>seq4',
 'gcggatttactcaggggagagcccagGGataaatggagtctgtgcgtccaca',
 'gaattcgcacca']

# Step 2: Define a Mapper Function

In [9]:
from collections import defaultdict

def process_FASTA_partition(iterator):
    hashmap = defaultdict(int)

    for fasta_record in iterator:
        if (fasta_record.startswith(">")):
            hashmap["z"] += 1
        else:
            chars = fasta_record.lower()
            for c in chars:
                hashmap[c] += 1
    #end-for

    print("hashmap=", hashmap)
    key_value_list = [(k, v) for k, v in hashmap.items()]
    print("key_value_list=", key_value_list)
    return  key_value_list

def debug_partition(iterator):
    print("type(iterator)=", type(iterator))
    print("begin partition ===")
    elements = []
    for x in iterator:
        elements.append(x)
    print("elements=", elements)
    print("end partition ===")

In [11]:
print(records_rdd.getNumPartitions())
pairs_rdd = records_rdd.mapPartitions(process_FASTA_partition)

1


# Step 3: Find Frequencies of DNA Letters

In [12]:
frequencies_rdd = pairs_rdd.reduceByKey(lambda x, y: x+y)
frequencies_rdd.collect()

[('z', 4), ('c', 61), ('g', 53), ('t', 45), ('a', 73), ('n', 2)]

In [14]:
frequencies_rdd.collect()

[('z', 4), ('c', 61), ('g', 53), ('t', 45), ('a', 73), ('n', 2)]

In [15]:
frequencies_rdd.collectAsMap()

{'z': 4, 'c': 61, 'g': 53, 't': 45, 'a': 73, 'n': 2}

Pros:

The provided solution works, simple, and semi-efficient. This solution improves on Version 1, by emitting much less (key, value) pairs, since we create a dictionary per input record and then flatten it into a list of (key, value) pairs, where key is a DNA-letter and value is an associated aggregated frequency of the DNA-letter.

Network traffic is improved by emitting much fewer (key, value) pairs.

There is no scalability issue since we use reduceByKey() for reducing all (key, value) pairs

Cons:

For each DNA sequence, this solution emits up to 6 (key, value) pairs, where key is a DNA-letter and value is sum of associated frequencies. This is a much improvement over solution version 1

Performance is not an optimal since we are still emitting about 6 (key, value) pairs per DNA string

This solution might be using too much memory due to creation of a dictionary per DNA sequence